In [ ]:
import flavio
import flavio.plots as fpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
obs = '<P5p>(B0->K*mumu)';

In [ ]:
flavio.Observable[obs].get_measurements()

In [ ]:
include_measurements=[
    'LHCb B->K*mumu 2020 P 0.1-0.98',
    'LHCb B->K*mumu 2020 P 1.1-2.5',
    'LHCb B->K*mumu 2020 P 2.5-4',
    'LHCb B->K*mumu 2020 P 4-6',
    'LHCb B->K*mumu 2020 P 15-19',
]

In [ ]:
bin_list=[(0.1,0.98), (1.1,2.5), (2.5,4), (4,6), (15,19)]

In [ ]:
fpl.bin_plot_exp(obs, include_measurements=include_measurements, col_dict={'LHCb': 'black'})

fpl.bin_plot_th(obs, bin_list=bin_list, N=50, threads=8, label='SM')

plt.legend()
plt.ylim(-1,1)
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

In [ ]:
from wilson import Wilson

w = Wilson({'C9_bsmumu':-1}, 4.8, 'WET', 'flavio')

In [ ]:
fpl.bin_plot_exp(obs, include_measurements=include_measurements, col_dict={'LHCb': 'black'})

fpl.bin_plot_th(obs, bin_list=bin_list, N=50, threads=8, label='SM')

fpl.bin_plot_th(obs, wc=w, bin_list=bin_list, N=50, threads=8, label='$C_9=-1$', fc='C2')

plt.legend()
plt.ylim(-1,1)
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

In [ ]:
from flavio.statistics.likelihood import Likelihood
from flavio.math.optimize import minimize

In [ ]:
measurments = [
    "BaBar RD 2012",
    "Belle B->D*lnu 2018 untagged",
    "Belle RD had 2015",
    "Belle RD* had 2016",
    "Belle RD* sl 2019",
    "Belle Rmue(B->D*lnu) 2017",
    "LHCb RD* 2015",
    "LHCb RD* 2017",
]
observables = [
    "Rtaul(B->Dlnu)",
    "Rmue(B->D*lnu)",
    "Rtaul(B->D*lnu)",
    "Rtaumu(B->D*lnu)",
]

In [ ]:
L = Likelihood(observables=observables,
               include_measurements=measurments)

In [ ]:
par = flavio.parameters.default_parameters.get_central_all()

def LL(x):
    Re_CVL, Re_CVR = x
    w = Wilson({'CVL_bctaunutau': Re_CVL, 'CVR_bctaunutau': Re_CVR},
                scale=4.8,
                eft='WET', basis='flavio')
    return L.log_likelihood(par, w, delta=True)

In [ ]:
def chi2_LL(x):
    return -2*LL(x)

In [ ]:
chi2_min_LL = minimize(chi2_LL,[0,0])
chi2_min_LL

In [ ]:
import flavio.plots as fpl

In [ ]:
%%time
cdat = fpl.likelihood_contour_data(LL,
                                   x_min=0, x_max=0.2, y_min=-0.2, y_max=0.2,
                                   n_sigma=(1, 2, 3),
                                   threads=8,
                                   steps=10)

In [ ]:
plt.figure(figsize=(10,6))
fpl.contour(**cdat, z_min=chi2_min_LL['fun'], interpolation_factor=9)
plt.xlabel(r'Re $C_{VL}$');
plt.ylabel(r'Re $C_{VR}$');